# Ingest an existing Dataset or Model

In this notebook we are going to showcase how to ingest an existing dataset or model into EOTDL.

Once it is ingested, you can use it in the same way as any other dataset or model in EOTDL (exploring, downloading, etc.).

## Ingesting through the CLI

The recommended way to ingest a dataset is using the CLI.

In [1]:
!eotdl datasets ingest --help

                                                                                
 Usage: eotdl datasets ingest [OPTIONS]                                         
                                                                                
 Ingest a dataset to the EOTDL.                                                 
 This command ingests the dataset to the EOTDL. The dataset must be a folder    
 with the dataset files, and at least a metadata.yml file or a catalog.json     
 file. If there are not these files, the ingestion will not work. All the files 
 in the folder will be uploaded to the EOTDL.                                   
                                                                                
 The following constraints apply to the dataset name:                           
 - It must be unique                                                            
 - It must be between 3 and 45 characters long                                  
 - It can only contain alpha

In order to ingest a dataset you will need a folder in your system with the data you want to upload.

In [2]:
!ls example_data

eurosat_rgb_dataset	 jaca_dataset	    jaca_dataset_stac_labels
eurosat_rgb_stac	 jaca_dataset_q2    labels_scaneo
eurosat_rgb_stac_labels  jaca_dataset_stac  sample_stacdataframe.csv


For this tutorial we are going to work with a subsample of the [EuroSAT](https://www.eotdl.com/datasets/EuroSAT-RGB) dataset.

In [3]:
from glob import glob 

files = glob('example_data/EuroSAT-small/**/*.*', recursive=True)
files

['example_data/EuroSAT-small/metadata.yml',
 'example_data/EuroSAT-small/Forest/Forest_3.tif',
 'example_data/EuroSAT-small/Forest/Forest_1.tif',
 'example_data/EuroSAT-small/Forest/Forest_2.tif',
 'example_data/EuroSAT-small/AnnualCrop/AnnualCrop_3.tif',
 'example_data/EuroSAT-small/AnnualCrop/AnnualCrop_1.tif',
 'example_data/EuroSAT-small/AnnualCrop/AnnualCrop_2.tif']

A `metadata.yml` file is required for Q0 datasets and models, containing some basic required information (dataset authors, licens, link to source and dataset name)

In [5]:
!cat example_data/EuroSAT-small/metadata.yml

authors:
- Patrick Helber
license: open
source: http://madm.dfki.de/downloads
name: EuroSAT-small


The chosen name is the one that will appear in the repository, hence it must be unique, between 3 and 45 characters long and can only contain alphanumeric characters and dashes (learn more at [https://www.eotdl.com/docs/datasets/ingest](https://www.eotdl.com/docs/datasets/ingest)).

Trying to ingest a dataset without a `metadata.yml` file will fail.

If everything is correct, the ingestion process should suceed.

In [7]:
!eotdl datasets ingest -p example_data/EuroSAT-small/

Uploading directory example_data/EuroSAT-small...
generating list of files to upload...
100%|███████████████████████████████████████████| 7/7 [00:00<00:00, 4106.31it/s]
No new files to upload


And now your dataset is avilable at EOTDL

In [8]:
!eotdl datasets list -n eurosat-small

['EuroSAT-small']


> Since the `EuroSAT-small` name is already taken, this process should fail for you. To solve it, just upload the dataset with a different name. However, this will polute the EOTDL with test datasets so we encourage you to try the ingestion process with a real dataset that you want to ingest (or overwrite your test dataset in the future with useful data).

In order to ingest Q1+ datasets, a valid STAC catalog is required instead of the `metadata.yml` file. We will explore this in the [data curation](tutorials/workshops/bids23/05_STAC_metadata.ipynb) notebook.

You can ingest a model exactly in the same way

In [9]:
!eotdl models ingest --help

                                                                                
 Usage: eotdl models ingest [OPTIONS]                                           
                                                                                
 Ingest a model to the EOTDL.                                                   
 This command ingests the model to the EOTDL. The model must be a folder with   
 the model files, and at least a metadata.yml file or a catalog.json file. If   
 there are not these files, the ingestion will not work. All the files in the   
 folder will be uploaded to the EOTDL.                                          
                                                                                
 The following constraints apply to the model name:                             
 - It must be unique                                                            
 - It must be between 3 and 45 characters long                                  
 - It can only contain alpha

## Versioning

By default, every time you re-upload a dataset or model a new version is created.

When you download a dataset, the latest version is used by default.

In [10]:
!eotdl datasets get EuroSAT-small

Dataset `EuroSAT-small v10` already exists at /home/juan/.cache/eotdl/datasets/EuroSAT-small/v10. To force download, use force=True or -f in the CLI.


However, you can specify the version

In [3]:
!eotdl datasets get EuroSAT-small -v 1

Dataset `EuroSAT-small v1` already exists at /home/juan/.cache/eotdl/datasets/EuroSAT-small/v1. To force download, use force=True or -f in the CLI.


In [4]:
!ls $HOME/.cache/eotdl/datasets/EuroSAT-small

v1  v10  v2  v3  v9


We apply versioning at dataset/model and file level, meaning only new or modified files will be uploaded in future re-uploads, downloading the appropriate files for each version.

You can explore the different versions in the user interface.

## Ingesting through the Library

You can ingest datasets and models using the library

In [11]:
from eotdl.datasets import ingest_dataset

ingest_dataset("example_data/EuroSAT-small");

Uploading directory example_data/EuroSAT-small...
generating list of files to upload...


100%|██████████| 7/7 [00:00<00:00, 6440.04it/s]


Exception: No new files to upload

## Ingesting through the API

Ingesting a dataset or model through the API is a multi step (and error prone) process:

1. Create/Retrieve a dataset
2. Create a version
3. Ingest files to version
	1. Ingest small files in batches
	2. Ingest large files in chunks as multipart upload
		1. Create multipart upload
		2. Ingest chunks
		3. Complete multipart upload
	3. Ingest existing files in batches to new version

The library/CLI will take care of these steps, so it is the recommended way to ingest a dataset. 

However, if you still want to ingest datasets with the API, we recommend following the previous steps using the API [documentation](https://api.eotdl.com/docs) or reading the implementation of the ingestion functions in the library. If you need further help, reach out to us at the Discord server.

This is a process we would like to simplify in the future.